# Number of collisions

In [1]:
%matplotlib widget 

# global import
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
# local import
import lppydsmc as ld

seed = 1111
np.random.seed(seed)

In [2]:
def number_of_collisions(mass, radius, quantity, time_step, iterations, temperature = 300, density = 3.2e19, cell_volume = 1e-9, seed = None):
    import pandas as pd
    import lppydsmc as ld
    import numpy as np
    from tqdm import tqdm
    from pathlib import Path
    np.random.seed(seed)
    collisions_list = []
    points = 1e-3*np.array([[0,0],[0,1],[1,1],[1,0]])
    positions = ld.initialization.particles.uniform_position(quantity, points)

    vel_std = ld.utils.physics.gaussian(temperature, mass)
    velocities = np.random.normal(loc=0.0, scale=vel_std, size = (quantity,3))
    
    arrays = [np.concatenate((positions, velocities), axis = 1)]
    
    grid = np.array([np.zeros((quantity,2))]) # 1 cell
    grid[0][:,1] = np.array([k for k in range(quantity)])
    currents = np.array([quantity])
    averages = np.array([quantity])
    
    cross_section = np.pi*4*radius*radius
    cross_sections = np.array([[cross_section]])

    max_proba = np.array([3*cross_section*ld.utils.physics.maxwellian_mean_speed(temperature, mass)])

    target_density = density
    cell_volume = cell_volume
    particles_weight = target_density*cell_volume/quantity
    remains_per_cell = np.array([0.])
    masses = np.array([mass])
    monitoring = None
    group_fn = None
    
    period_saving = 10

    for iteration in tqdm(range(1,iterations+1)) :
        
        results = ld.collision.handler_particles_collisions(arrays, grid, currents, time_step, \
            averages, max_proba, cross_sections, cell_volume, particles_weight, remains_per_cell, masses, monitoring = monitoring, group_fn = group_fn)
        collisions_list.append(results)
        # if(iteration%100 == 0):
        #    print(ld.collision.collider.candidates(currents, time_step, averages, max_proba, cell_volume, particles_weight, remains_per_cell))
        #    print(remains_per_cell)
        #     fig, ax = plt.subplots(3)
        #     ax[0].hist(arrays[0][:,2],color = 'r', bins = 30)
        #     ax[1].hist(arrays[0][:,3], color = 'g', bins = 30)
        #     ax[2].hist(arrays[0][:,4], color = 'b', bins = 30)
    return collisions_list

In [11]:
mass = 2.16e-25 # kg
time_step = 1e-5
radius = 2e-10
J_TO_EV = 1/1.6e-19
iterations = 1000
T = 300
n = 3.2e19
quantity = int(1e5)
cell_volume = 1e-9
collisions_list = number_of_collisions(mass = mass, radius = radius, quantity = quantity, time_step = time_step, \
                                       iterations = iterations, temperature = T, density= n,cell_volume = cell_volume, seed = 1111)

100%|██████████| 1000/1000 [01:18<00:00, 12.67it/s]


In [12]:
fig, ax = plt.subplots()
ax.plot(collisions_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
BOLTZMAN_CONSTANT = 1.38064e-23 # J K−1

def relative_speed_maxwellian(temperature,mass):
    return 4/np.sqrt(np.pi)*np.sqrt(BOLTZMAN_CONSTANT*temperature/mass)
# 1st : theoretical collision frequency for MB distribution
def number_of_collisions_per_time_and_volume(T, m, d, n): # per unit of volume and time
    cross_section = np.pi*d*d
    return 1/2 * cross_section *n**2 * relative_speed_maxwellian(T,m)

In [14]:
relative_speed_maxwellian(T, mass)

312.50648393656974

In [15]:
nu_c = number_of_collisions_per_time_and_volume(T = T, m = mass, d=2*radius, n = n)
simu_time = iterations*time_step
volume_system = 1e-9
particles_weight = n*volume_system/quantity

In [16]:
particles_weight

320000.00000000006

In [17]:
collisions_nb = np.sum(np.array(collisions_list)) # we count twice too many 
print('Total number of collision : {:e}'.format(collisions_nb));
print('Equivalent in reality : {:e}'.format(particles_weight*collisions_nb))
print('Expected number of collisions in reality : {:e}'.format(nu_c*simu_time*volume_system))
print('Ratio of the two : {:.2}'.format(100*(particles_weight*collisions_nb-nu_c*simu_time*volume_system)/(nu_c*simu_time*volume_system)))

Total number of collision : 2.511540e+06
Equivalent in reality : 8.036928e+11
Expected number of collisions in reality : 8.042644e+11
Ratio of the two : -0.071
